In [13]:
from modelscope import BertTokenizer, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('AI-ModelScope/bert-base-uncased')
model = BertForMaskedLM.from_pretrained('AI-ModelScope/bert-base-uncased')
model.eval()

2025-05-04 20:25:19,266 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


2025-05-04 20:25:19,900 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


Some weights of the model checkpoint at /root/.cache/modelscope/hub/models/AI-ModelScope/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [16]:
# check
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
print(output)

MaskedLMOutput(loss=None, logits=tensor([[[ -9.3893,  -9.3195,  -9.1902,  ...,  -8.1698,  -8.5670,  -5.9066],
         [-10.8334, -10.7239, -10.7502,  ...,  -9.9730,  -9.6572,  -8.2451],
         [-11.6256, -11.6355, -11.4426,  ..., -10.0851, -11.0916,  -8.9086],
         ...,
         [ -8.9649,  -8.7341,  -9.0515,  ...,  -9.3791,  -9.9774,  -6.0243],
         [-10.7885, -10.6272, -10.9118,  ...,  -8.5793, -10.5830,  -5.4193],
         [-10.3052, -11.3263, -10.4285,  ...,  -9.6418,  -9.9589,  -7.2134]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)


In [18]:
import json

with open('data/dataset.jsonl', 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

In [19]:
from utils import Score
scorer = Score(bert_model=model, bert_tokenizer=tokenizer)

In [22]:
correct = 0
total = len(data)

for item in data:
    sentence0 = item['sentence0']
    sentence1 = item['sentence1']
    is_false = item['false']  # 0 means sentence0 is false, 1 means sentence1 is false

    score0 = scorer.bert_score(sentence0)
    score1 = scorer.bert_score(sentence1)

    predicted_true = 0 if score0 < score1 else 1

    if predicted_true != is_false:
        correct += 1

accuracy = correct / total
print(f"BERT Sense-Making Task Accuracy: {accuracy:.4f}")

BERT Sense-Making Task Accuracy: 0.6957
